# Predicting churn with the SPSS random tree algorithm

This Scala 2.10 notebook shows you how to create a predictive model of churn rate by using IBM SPSS Algorithm on Apache Spark version 1.6. You'll learn how to create an SPSS random tree model by using the IBM SPSS Machine Learning API, and how to view the model with IBM SPSS Model Viewer.

Because it consists of multiple classification and regression trees (CART), you can use random tree algorithms to generate accurate predictive models and solve complex classification and regression problems. Each tree develops from a bootstrap sample that is produced by resampling the original data points with replacement data. During the resampling phase, the best split variable is selected for each node from a specified smaller number of variables that are drawn randomly from the full set of variables. Each tree grows without pruning and then, during the scoring phase, the random tree algorithm aggregates tree scores by majority voting (for classification) or average (for regression).

In this notebook, you'll create a model with telecommunications data to predict when its customers will leave for a competitor, so that you can take some action to retain the customer.
    
To get the most out of this notebook, you should have some familiarity with the Scala programming language.

## Contents 
This notebook contains the following main sections:

1. [Load the Telco Churn data to the cloud data repository.](#overview)
1. [Prepare the data.](#prepare)
1. [Configure the RandomTrees model.](#configure) 
1. [View the model.](#view)
1. [Summary and next steps.](#next)    

<a id="overview"></a>
## 1. Load the Telco Churn data to the cloud data repository.
Telco Churn is a hypothetical data file that concerns a telecommunications company's efforts to reduce turnover in its customer base. Each case corresponds to a separate customer and it records various demographic and service usage information. Before you can work with the data, you must use the URL to get the telco.csv and telco_Feb.csv files from the GitHub repository.


In [10]:
val link_telco = "https://raw.githubusercontent.com/AlgorithmDemo/SampleData/master/telco.csv"

import sys.process._
import java.net.URL
import java.io.File
new URL(link_telco) #> new File("telco.csv") !!

val link_telco_Feb = "https://raw.githubusercontent.com/AlgorithmDemo/SampleData/master/telco_Feb.csv"

import sys.process._
import java.net.URL
import java.io.File
new URL(link_telco_Feb) #> new File("telco_Feb.csv") !!

<a id="prepare"></a>
## 2. Prepare the data.

After uploading the CSV files that contain the data, you must create a SQLContext, put the data from the telco.scv file into a Spark DataFrame, and show the first row in the DataFrame.

In [2]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

val dfTelco = sqlContext.
    read.
    format("com.databricks.spark.csv").
    option("header", "true").
    option("inferschema", "true").
    load("telco.csv")

dfTelco.show(1)

Review the data. Print the schema of the DataFrame to look at what kind of data you have.

In [3]:
dfTelco.printSchema

Create a DataFrame for the telco_Feb.csv data. You'll use this year's data to build the model, and use the February data for accuracy value.

In [4]:
val dfTelcoFeb = sqlContext.
    read.
    format("com.databricks.spark.csv").
    option("header", "true").
    option("inferschema", "true").
    load("telco_Feb.csv")

<a id="configure"></a>
## 3. Configure the RandomTrees model.

By running this portion of the code, you create the random trees estimator, import the libraries, and set the ordinal and nominal variables. Because no inputFieldList value is set, all fields except the target, frequency, and analysis weight fields are treated as input fields. To make the random tree model build faster, set the number of trees to 10 instead of the default value, which is 100. Finally, you must specify the churn target field. 

In [5]:
import com.ibm.spss.ml.classificationandregression.ensemble.RandomTrees
import com.ibm.spss.ml.utils.DataFrameImplicits.DataFrameEnrichImplicitsClass

val ordinal = Array("ed")
val nominal = Array("region",
     "marital",
     "retire",
     "gender",
     "tollfree",
     "equip",
     "callcard",
     "wireless","multline",
     "voice","pager","internet","callid","callwait","forward","confer",
     "ebill",
     "custcat",
     "churn"
   )
val srf = RandomTrees().setTargetField("churn").setNumTrees(10)
val srfModel = srf.fit(dfTelco.setNominalMeasure(nominal,true).setOrdinalMeasure(ordinal,true))

Do the prediction and get your results.

In [6]:
val predResult = srfModel.transform(dfTelcoFeb)
val predResultNew = predResult.withColumn("prediction", predResult("prediction").cast("double")).
    withColumn("churn", predResult("churn").cast("double"))

To get the accuracy result, use the Apache Spark **MulticlassClassificationEvaluator** function. Notice that the accuracy is above 90%.

In [7]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
val evaluator = new MulticlassClassificationEvaluator().setLabelCol("churn").setMetricName("precision")
val acc_result = evaluator.evaluate(predResultNew)
println(s"acc_result:$acc_result")

<a id="view"></a>
## 4. View the model.

### Show the random trees model result.
To see the result, import the IBM SPSS Model Viewer, which you can use to explore different views of the model.

In [8]:
import com.ibm.spss.scala.ModelViewer
kernel.magics.html(ModelViewer.toHTML(srfModel))

### Export the XML files (PMML, StatXML) for other detail statistics.
By exporting your results to different formats, such as Predictive Model Markup Language (PMML) or statXML format you can share your statistical analyses outside of IBM Data Science Experience.

In [9]:
import java.io.{File, PrintWriter}

srfModel.toPMML("randomTrees_pmml.xml")
val statXML = srfModel.statXML()
new PrintWriter("StatXML.xml") {
      write(statXML)
      close
}

<a id="next"></a>
# Summary and next steps
You have created a predictive model of churn rate by using IBM SPSS Algorithm on Apache Spark. Now you can create a different model to compare model evaluations, such as the test of model effects, residuals, and so on. See [SPSS documentation](https://apsportal.ibm.com/docs/content/kc_gen/integrations-gen2.html).

## Authors

Wang Zhiyuan and Yu Wenpei are SPSS Algorithm Engineers at IBM.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.